In [1]:
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import os,cv2
import torch.nn as nn
from cyvlfeat.kmeans import kmeans,kmeans_quantize
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
uncertianty=True


In [2]:
feats=np.load('record_feats_train.npy').astype(np.float32)
data=np.reshape(feats,(feats.shape[0],-1))

if  not os.path.exists('sparse_labels.npy'):
    print('clustering')
    K=10
    centers=kmeans(data,num_centers=K,initialization='PLUSPLUS',num_repetitions=10,
                   max_num_comparisons=100,max_num_iterations=100,algorithm='LLOYD',num_trees=3)
    labels=kmeans_quantize(data,centers)
    # to get the sparse matrix of labels
    sparse_labels=np.eye(K)[labels]
    labels=labels.astype(np.float32)
    sparse_labels=sparse_labels.astype(np.float32)

    np.save('sparse_labels.npy',sparse_labels)
    np.save('labels.npy',labels)
else:
    sparse_labels=np.load('sparse_labels_.npy').astype(np.float32)
    labels=np.load('labels_.npy').astype(np.float32)


In [3]:
BATCH_SIZE=64
train_data_loadpath='/mnt/sda/sunche/datasets/Anomaly/Avenue/train/'
train_label_loadpath='/mnt/sda/sunche/datasets/Anomaly/Avenue/train/'
root='/mnt/sda/sunche/datasets/Anomaly/Avenue/'
def Gaussian(X,rate=0.05):
    # Salt and pepper noise
    drop = np.random.normal(0,rate, X.shape)
    X=X+drop
    return X
class MyDataset(Dataset):
    def __init__(self, train_data_loadpath,labels,transform=None, target_transform=None,):
        self.path=train_data_loadpath
        self.files=os.listdir(train_data_loadpath)
        self.inds=range(len(self.files))
        self.labels=labels
    def __getitem__(self, index):
        img_path=os.path.join(self.path,str(self.inds[index]+1)+'.jpg')
        img = cv2.resize(cv2.imread(img_path),(224,224))
        img_npy=np.array(img).astype(np.float32)/255.0
        
#         img_npy = Gaussian(img_npy)
        img_npy = np.reshape(img_npy,[224,224,3])
        img_npy = np.transpose(img_npy,[2,0,1])
        label=self.labels[index]

        img_npy=torch.from_numpy(img_npy).float()
#         label=torch.from_numpy(label).float()
        return img_npy,label
    def __len__(self):
        return len(self.files)
    
# torch.set_num_threads(1)
# train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE,num_workers=16, shuffle=True,pin_memory=False)   


# class MyDataset(Dataset):
#     def __init__(self, feats, labels):
#         self.feats=feats
#         self.labels=labels
#     def __getitem__(self, index):
#         return self.feats[index],self.labels[index]
#     def __len__(self):
#         return len(self.feats)
BATCH_SIZE=64
if uncertianty==False:
    train_data=MyDataset(train_data_loadpath,labels)
if uncertianty==True:
    train_data=MyDataset(train_data_loadpath,sparse_labels)

torch.set_num_threads(1)
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE,num_workers=16, shuffle=True,pin_memory=False)  

In [4]:
num_inputs, num_outputs, num_hiddens = data.shape[1], 10, 256
import auto_encoders
model = auto_encoders.VGG_11(20)

In [5]:
if uncertianty==False:
    epoch_=100
    model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
    model.cuda()
    optimizer = torch.optim.Adam(model.parameters(),lr=1e-3,weight_decay=0)
    loss_func = nn.CrossEntropyLoss() 
    loss_func.cuda() 
    num=0
    model.train()
    record_loss=[]
    for epoch in range(epoch_):
        print('epoch {}'.format(epoch + 1))
        # training-----------------------------
        train_loss = 0.
        for i,(batch_x, batch_y) in enumerate(train_loader):
          batch_x = batch_x.cuda()
          batch_y = batch_y.cuda()
          model.zero_grad()
        #           print(batch_x.size())
          out= model(batch_x)
          _, preds = torch.max(out, 1)
          loss = loss_func(out, batch_y.long())
          train_loss += loss.item()
          loss.backward()
          optimizer.step()
          print('epoch:{},batchID{},RLoss{:.6f}'.format(epoch,i,loss.item()))
        if epoch%10==9:
          state = {'epoch': epoch+1,
                   'model_state': model.state_dict(),
                   'optimizer_state' : optimizer.state_dict(),}
          torch.save(state, './savepath_lstm_Net_6/cls_model'+str(epoch+1)+'.pkl')
          record_loss.append(train_loss)
        print('Train Loss: {:.6f}'.format(train_loss / (len(train_data))))
    print(record_loss)


In [6]:
# from helpers import get_device, one_hot_embedding
from losses import relu_evidence
from losses import edl_mse_loss, edl_digamma_loss, edl_log_loss, relu_evidence
criterion = edl_digamma_loss
if uncertianty==True:
    epoch_=30
    model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
    model.cuda()
    optimizer = torch.optim.Adam(model.parameters(),lr=1e-4,weight_decay=0.005)
    loss_func = nn.CrossEntropyLoss() 
    loss_func.cuda() 
    num=0
    model.train()
    record_loss=[]
    exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(
            optimizer, step_size=7, gamma=0.1)
    for epoch in range(epoch_):
        print('epoch {}'.format(epoch + 1))
        # training-----------------------------
        train_loss = 0.
        for i,(batch_x, batch_y) in enumerate(train_loader):
          batch_x = batch_x.cuda()
          batch_y = batch_y.cuda()
          model.zero_grad()
        #           print(batch_x.size())
          out= model(batch_x)
          _, preds = torch.max(out, 1)
#           print(out.size(),batch_y.size())
          loss = criterion(out, batch_y.float(), epoch, 20, 10)
          evidence = relu_evidence(out)
          alpha = evidence + 1
          u = 20 / torch.sum(alpha, dim=1, keepdim=True)
          train_loss += loss.item()

          loss.backward()
          optimizer.step()
#           print('epoch:{},batchID{},RLoss{:.6f}'.format(epoch,i,loss.item()))
        exp_lr_scheduler.step()
        if epoch%1==0:
          state = {'epoch': epoch+1,
                   'model_state': model.state_dict(),
                   'optimizer_state' : optimizer.state_dict(),}

          torch.save(state, './savepath_lstm_Net_6/cls_evi_model'+str(epoch+1)+'.pkl')
          record_loss.append(train_loss)
        print('Train Loss: {:.6f}'.format(train_loss / (len(train_data))))
    print(record_loss)


epoch 1
Train Loss: 0.019392
epoch 2
Train Loss: 0.016296
epoch 3
Train Loss: 0.015746
epoch 4
Train Loss: 0.015365
epoch 5
Train Loss: 0.015231
epoch 6
Train Loss: 0.014873
epoch 7
Train Loss: 0.014580
epoch 8
Train Loss: 0.012185
epoch 9
Train Loss: 0.011459
epoch 10
Train Loss: 0.011082
epoch 11
Train Loss: 0.010806
epoch 12
Train Loss: 0.010607
epoch 13
Train Loss: 0.010309
epoch 14
Train Loss: 0.010076
epoch 15
Train Loss: 0.009701
epoch 16
Train Loss: 0.009507
epoch 17
Train Loss: 0.009455
epoch 18
Train Loss: 0.009374
epoch 19
Train Loss: 0.009367
epoch 20
Train Loss: 0.009299
epoch 21
Train Loss: 0.009291
epoch 22
Train Loss: 0.009217
epoch 23
Train Loss: 0.009220
epoch 24
Train Loss: 0.009222
epoch 25
Train Loss: 0.009217
epoch 26
Train Loss: 0.009206
epoch 27
Train Loss: 0.009192
epoch 28
Train Loss: 0.009204
epoch 29
Train Loss: 0.009187
epoch 30
Train Loss: 0.009188
[297.3212262392044, 249.85754197835922, 241.41895109415054, 235.5831083059311, 233.52069109678268, 228.030629